<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/Homework_5/hw5_sqp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# importing all the libraries
import numpy as np
from matplotlib import pyplot as plt
import torch as t

In [17]:
# defining the objective function as well as inequality constraints
function = lambda x: x[0]**2 + (x[1] - 3)**2
g_1 = lambda x: x[1]**2 - 2*x[0]
g_2 = lambda x: (x[1]-1)**2 + 5*x[0]-15

grad_fun = lambda x : t.tensor([[2 * x[0], 2 * (x[1] - 3)]])
grad_g_1 = lambda x : t.tensor([[-2, 2 * x[1]]])
grad_g_2= lambda x: t.tensor([[5, 2 * (x[1] - 1)]])

g = lambda x: t.tensor([g_1(x), g_2(x)])
grad_g = lambda x: t.tensor[grad_g_1(x), grad_g_2(x)]

In [18]:
# function for updating weights for the merit function
def update_weights(mu, w, iter):
  if iter>0:
    w = t.max(abs(mu), 0.5 *(w + abs(mu)))
  else:
    w = abs(mu)
  return w

# function which calculates F(alpha)
def fun_alp(x, w, alpha, s):
  new_fun = function(x + alpha*s) +  w[0,:] * max(0, g_1(x + alpha*s)) + w[1,:]* max(0, g_2(x + alpha*s))
  return new_fun

In [19]:
# funciton for calculating the armijo linesearch algorithm
def line_Search(x, mu, w, s, iter):
  t = 0.5
  alpha = 1
  w = update_weights(mu, w, iter)
  if g_1(x) <= 0:
    Dg1Da = 0
  else:
    Dg1Da = t.matmul(grad_g_1(x), s)

  if g2(x) <= 0:
    Dg2Da = 0
  else:
    Dg2Da = t.matmul(grad_g_1(x), s)
  
  dfun_da = t.matmul(grad_fun(x), s) + (w[0, :] * Dg1Da + w[1, :] * Dg2Da)

  phi = lambda x, t, alpha, w, dfun_da: fun_alp(x, w, 0, 0) + t * alpha * dfun_da

  while phi(x, t, alpha, w, dfun_da) < fun_alp(x, w, alpha, s):
    alpha = alpha/2
  return alpha, w

In [20]:
# function for approximating the Hessian via Broyden-Fletcher-Goldfarb-Shanno(BFGS) Method
def bfgs(x, Lxx, s, mu, alpha):
  grad_l_k = grad_fun(x) + t.matmul(mu.T, g(x))
  x_up = x + s *alpha 
  grad_l_kp1 = grad_fun(x) + t.matmul(mu.T, g(x_up))

  delt_l = grad_l_kp1 - grad_l_k

  Q = t.matmul((t.matmul(alpha*s).T, Lxx), (alpha*s))
  if t.matmul((alpha*s).T, delt_l.T) >= 0.2 * t.matmul(t.matmul((alpha*s).T, Lxx), (alpha*s)):
    th = 1
  else:
    th = 0.8*Q / (Q-t.matmul((alpha*s).T, delt_l.T))
  y = th * delt_l.T + (1-th)*t.matmul(Lxx, (alpha*s))
  Lxx = Lxx + t.matmul(y, y.T) / t.matmul(y.T, s) - t.matmul(t.matmul(Lxx, s), t.matmul(s.T, Lxx))/t.matmul(t.matmul(s.T, Lxx), s)
  return Lxx

In [21]:
def check_lag_multiplier(mu, act_set):
  mu_check = 0
  if len(mu) == 0 or min(mu) > 0:
    mu_check = 1
  else:
    mu_idx = np.argmin(np.array(mu))
    mu = mu[mu!=min(mu)]
    act_set.pop(mu_idx)
  return act_set, mu_check, mu

In [22]:
# defining the QP subproblem solver
def sol_qp(x, Lxx):
  act_set = []
  init_A = t.cat((grad_g_1, grad_g_2), 0)
  init_B = t.tensor([[g_1(x), g_2(x)]]).T
  init_mu = t.zeros((init_B.shape[0], 1))
  mu = []
  while True:
    if len(act_set) == 0:
      s_mu = t.matmul(t.linalg.inv(Lxx), -grad_fun(x).T)
      s = s_mu[:2, :1]
    if len(act_set) > 0:
      if len(act_set) == 1:
        A = init_A[act_set[0], :].reshape(1,-1)
        B = init_B[act_set[0], :].reshape(1, 1)
      if len(act_set) == 2:
        A = init_A
        B = init_B
      
      z = t.zeros((A.shape[0], A.shape[0]))
      mat = t.cat((t.cat((Lxx, A.T), 1), t.cat((A, z), 1)), 0)
      j = t.cat((-grad_fun(x).T, -B), 0)
      s_mu = t.matmul(t.linalg.inv(mat), j)
      s = s_mu[:2, :]
      mu = s_mu[2:, :]
    
    if len(mu)==1:
      init_mu[0] = s_mu[2:3, :]
    if len(mu)==2:
      init_mu[0] = s_mu[2:3, :]
      init_mu[1] = s_mu[3:, :]

    const_sqp = t.round((t.matmul(init_A, s.reshape(-1,1)) + init_B))
    act_set, che_mu, mu = check_lag_multiplier(mu, act_set)

    if t.max(const_sqp) <=0 and che_mu==1:
      return s, init_mu
    else:
      indx = np.argmax(const_sqp)
      act_set.append(indx)
      act_set = np.unique(np.array(act_set)).tolist()